Домашнее задание №7 по курсу «Машинное обучение»: SGD

Скачайте датасет «Ирисы Фишера» (http://archive.ics.uci.edu/ml/datasets/Iris). В этих данных описаны три вида ирисов. Случайным образом разбейте датасет в соотношении 9 : 1. Первую
часть надо будет использовать в качестве обучающей выборки. Вторую — в качестве тестовой (для
оценки качества полученной гипотезы). В каждом задании вам необходимо будет построить три модели, по одной модели для каждого вида ирисов. Соответствующая модель должна решать задачу
классификации на два класс — принадлежит данный цветок рассматриваемому виду или нет.
1. обучите модель логистической регрессии с регуляризацией Тихонова с помощью SGD-алгоритма.
Необходимые параметры подберите с помощью алгоритма k-fold.
2. обучите модель логистической регрессии с регуляризацией Тихонова с помощью алгоритма batch
gradient descent. Этот алгоритм выбирает в качестве направления на каждом шаге −∇LS(w(t)).
Необходимые параметры подберите с помощью алгоритм k-fold.
3. сравните два полученных решения. Что вы можете сказать о плюсах и минусах каждого из подходов?


In [2]:
import pandas as pd
import io
import requests
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [3]:
# Load data from url in list
def load_data():
    url="http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    s=requests.get(url).content
    data=pd.read_csv(io.StringIO(s.decode('utf-8')),index_col=False,header=None)
    dict  = {4:{'Iris-setosa':0,'Iris-versicolor':1,'Iris-virginica':2}}
    data.replace(to_replace=dict,inplace=True)
    return  data.values.tolist()


# Split a dataset into train and test set
def split_train_test(dataset, train_ratio,test_ratio):
    sets = []
    train_size = int( train_ratio * len(dataset) / ( train_ratio + test_ratio)) 
    for i in range(3):
        set = []
        for x in dataset:
            if x[-1]==i:
                v = x[:-1]
                v.append(-1)
            else:
                v = x[:-1]
                v.append(1)
            set.append(v)
        np.random.shuffle(set)
        train, test = set[:train_size], set[train_size:]   
        sets.append((train,test))
    return sets
       

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    fold_size = int(len(dataset) / n_folds)
    np.random.shuffle(dataset)
    splits = []
    for i in xrange(n_folds):
        part_before = dataset[:fold_size*i]
        part_after = dataset[fold_size*(i + 1):]
        validation_part = dataset[fold_size*i : fold_size*(i + 1)]
        training_part = part_before + part_after
        splits.append((training_part, validation_part))
    return splits


In [4]:
def predict(w, x):
    return 1/(1 + np.exp(-np.dot(w,x)))


def predict_class(w, x):
    return -1 if predict(w,x) < 0.5 else 1


def get_risk(predictions, true_labels):
    wrong = np.sum([1 for i,j in zip(predictions,true_labels) if i!=j])
    return float(wrong) / len(true_labels)


def dLoss(w, lamda, x, y):
    return -np.dot(y,x)/(np.exp(y*np.dot(w,x)) + 1) + 2*np.dot(lamda,w)

In [5]:
def SGD(T,lamda, eta, X, y):
    W = []
    N = len(X)
    W.append(np.zeros(shape=(1,len(X[0]))))
    for t in xrange(T):
        idx = np.random.choice(N,1)
        vt = dLoss(W[t], lamda, X[idx], y[idx])
        W.append(W[t] - eta*vt)
        t += 1
    for i in xrange(1,T):
        W[0] += W[i]
    return W[0]/T


def BGD(T,lamda, eta, X, y, batch_size = None):
    W = []
    N = len(X)
    W.append(np.zeros(shape=(1,len(X[0]))))
    for t in xrange(T):
        vt = np.sum(dLoss(W[t], lamda, X[i], y[i]) for i in xrange(N))
        vt /= N
        W.append(W[t] - eta*vt)
    for i in xrange(1,T):
        W[0] += W[i]
    return W[0]/T


Для каждой модели подберем параметры: T, шаг спуска и коэффициент регуляризации на 5-folds. 

In [49]:
irisNames = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
dataset = load_data()
sets = split_train_test(dataset,9,1)

LAMBDAS = np.linspace(0.0, 0.99 , 5)
ETAS = [.001, .003, .01, .03, .1, .3]
MODELS = [SGD,BGD]
T=[50,100,150,200]


def param_tunning(MODELS, T, LAMBDAS, ETAS):
    
    best_params = []
    
    for model in MODELS:
        print ('Running model for ' + model.__name__)
        for idx in xrange(3):
            print ('Running model for ' + irisNames[idx])
            train, test = sets[idx]
            splits = cross_validation_split(train, 5)
            params = []
            print ("Running 5-folds for tunning T, lamda, eta... ")
            for t in T:
                for lamda in LAMBDAS:
                    for eta in ETAS:
                        risk = 0
                        for split in splits:
                            train = split[0]
                            split_X_train = [sublist[:4] for sublist in train]
                            split_y_train = [sublist[-1] for sublist in train]

                            validation = split[1]
                            split_X_validation = [sublist[:4] for sublist in validation]
                            split_y_validation = [sublist[-1] for sublist in validation]

                            fitted_model = model(t, lamda, eta, split_X_train, split_y_train)
                            predicted = [predict_class(fitted_model, x) for x in split_X_validation]
                            risk += get_risk(predicted, split_y_validation)

                        risk /= len(splits)

                        print ('T = %.3f\tLambda = %.3f\tEta = %.3f\tRisk = %.3f'%(t,lamda,eta,risk))
                        params.append((t,lamda,eta,risk))

                        if risk == 0:
                            break

                    if risk == 0:
                        break
                        
                if risk == 0:
                        break
                        
            best_param = min(params, key = lambda t: t[3])
            best_params.append((model.__name__,idx,best_param[0],best_param[1],best_param[2],best_param[3]))
            print ('%s: Best T = %.3f\tBest lambda = %.3f\t Best eta = %.3f\tRisk = %.3f'%(model.__name__,best_param[0],best_param[1],best_param[2],best_param[3]))
    return best_params

best_params = param_tunning(MODELS,T,LAMBDAS,ETAS)


Running model for SGD
Running model for Iris-setosa
Running 5-folds for tunning T, lamda, eta... 
T = 50.000	Lambda = 0.000	Eta = 0.001	Risk = 0.341
T = 50.000	Lambda = 0.000	Eta = 0.003	Risk = 0.341
T = 50.000	Lambda = 0.000	Eta = 0.010	Risk = 0.333
T = 50.000	Lambda = 0.000	Eta = 0.030	Risk = 0.274
T = 50.000	Lambda = 0.000	Eta = 0.100	Risk = 0.000
SGD: Best T = 50.000	Best lambda = 0.000	 Best eta = 0.100	Risk = 0.000
Running model for Iris-versicolor
Running 5-folds for tunning T, lamda, eta... 
T = 50.000	Lambda = 0.000	Eta = 0.001	Risk = 0.341
T = 50.000	Lambda = 0.000	Eta = 0.003	Risk = 0.341
T = 50.000	Lambda = 0.000	Eta = 0.010	Risk = 0.341
T = 50.000	Lambda = 0.000	Eta = 0.030	Risk = 0.319
T = 50.000	Lambda = 0.000	Eta = 0.100	Risk = 0.348
T = 50.000	Lambda = 0.000	Eta = 0.300	Risk = 0.437
T = 50.000	Lambda = 0.247	Eta = 0.001	Risk = 0.341
T = 50.000	Lambda = 0.247	Eta = 0.003	Risk = 0.341
T = 50.000	Lambda = 0.247	Eta = 0.010	Risk = 0.341
T = 50.000	Lambda = 0.247	Eta = 0.03

In [50]:
best_params

[('SGD', 0, 50, 0.0, 0.1, 0.0),
 ('SGD', 1, 50, 0.0, 0.03, 0.31851851851851853),
 ('SGD', 2, 150, 0.0, 0.1, 0.08148148148148147),
 ('BGD', 0, 50, 0.0, 0.1, 0.0),
 ('BGD', 1, 50, 0.0, 0.001, 0.34074074074074073),
 ('BGD', 2, 150, 0.0, 0.3, 0.037037037037037035)]

In [51]:
for i in xrange(len(best_params)):
    
    model, irisIdx , T, lamda, eta, risk = best_params[i]
    train, test = sets[irisIdx]
    
    X_train = [sublist[:4] for sublist in train]
    y_train = [sublist[-1] for sublist in train]
    X_test = [sublist[:4] for sublist in test]
    y_test = [sublist[-1] for sublist in test]
    
    fittedModel = locals()[model]( T , lamda, eta, X_train, y_train)
    predicted = [predict_class(fittedModel, x) for x in X_test]
    d_risk = get_risk(predicted, y_test)
    print ('Test risk of model %s on %s = %.3f' % (model,irisNames[irisIdx],d_risk))
    

Test risk of model SGD on Iris-setosa = 0.000
Test risk of model SGD on Iris-versicolor = 0.333
Test risk of model SGD on Iris-virginica = 0.000
Test risk of model BGD on Iris-setosa = 0.000
Test risk of model BGD on Iris-versicolor = 0.267
Test risk of model BGD on Iris-virginica = 0.000


Ошибки на тестовой выборке для BGD и SGD  одинаковые. Однако, для ошибка классификации Iris-versicolor  от остальных 2х классов  на тестовой выборке для SGD больше , чем для BGD при одинаковом количестве итераций. <br/>
В алгоритме BGD приходится на каждой итерации считать  градиент на всем датасете , и если датасет довольно большой , то этот процесс может быть очень трудоемким. Также необходимо чтобы весь датасет помещался в оперативной памяти. Но, учитывая то, что считается градиент на всех элементах, то спуск идет намного быстрее к минимому и требуется меньше итераций, чем SGD.<br/>
В SGD  на каждой итерации градиент считается только на одном элементе. Т.е. SGD более быстрый, однако приходится расплачиваться тем, что итераций будет больше. Плюс: позволяет добавлять новые данные в режиме «онлайн»